In [1]:
import numpy as np

# Image data types and what they mean

In `skimage`, images are simply `numpy` arrays, which support a variety of data types [1](https://numpy.org/doc/stable/user/basics.types.html), i.e. "dtypes". To avoid distorting image intensity (see Rescaling intensity values), we assume that images use the following ranges:

| Data type   |      Range      |  
|:----------|:-------------|
| uint8 |  0 to 255 | 
| uint16 |    0 to 65535   | 
| uint32 | 0 to 2^32 - 1 | 
| float |  -1 to 1 or 0 to 1 | 
| int8 |    -128 to 127   | 
| int16 | -32768 to 32767 |
| int32 | -2^31 to 2^31 - 1 |

Note that float images should be restricted to the range -1 to 1 even though the data type itself can exceed this range; all integer dtypes, on the other hand, have pixel intensities than can span the entire data type range. With a few exceptions, *64-bit (u)int images are not supported*.

Functions in `skimage` are designed so that they accept any of these dtypes, but, for efficinecy, *may return an image of a different dtype* (see Output types). If you need a particular dtype, `skimage` provides utility functions that convert dtypes and properly rescale image intensities (see Input types). You should **neven use** `astype` on an image, because it violates these assumptions about the dtype range:

In [2]:
from skimage.util import img_as_float
image = np.arange(0, 50, 10, dtype=np.uint8)
print(image.astype(np.float)) # These float values are out of range.

print(img_as_float(image))

[ 0. 10. 20. 30. 40.]
[0.         0.03921569 0.07843137 0.11764706 0.15686275]


## Input types

Although we aim to preserve the data range and type of input images, functions may support only a subset of these data-types. In such a case, the input will be converted to the required type (if possible), and warning message printed to the log o=if a memory copy is needed. Type requirements should be noted in the docstrings. 

The following utility functions in the main package are available to developers and users:

| Function name   |      Description      |  
|:----------|:-------------|
| img_as_float | Convert to 64-bit floating point. | 
| img_as_ubyte |    Convert to 8-bit uint.   | 
| img_as_uint | Convert to 16-bit uint. | 
| img_as_int | Convert to 16-bit int. | 

These functions convert images to the desired dtype and *properly rescale their values*:

In [3]:
from skimage.util import img_as_ubyte
image = np.array([0,0.5,1],dtype=float)
img_as_ubyte(image)

array([  0, 128, 255], dtype=uint8)

Be careful! These conversions can result in a loss of precision, since 8 bits cannot hold the same amount of information as 64 bits:

In [4]:
image = np.array([0,0.5,0.503,1],dtype=float)
img_as_ubyte(image)

array([  0, 128, 128, 255], dtype=uint8)

Additionally, some functions take a `preserve_range` argument where a range conversion is convenient but not necessary. For example, interpolation in `transform.warp` requires an image of typ float, whch should have a range in [0,1]. So, by default, input images will be rescaled to this range. However, in some cases, the image values represent physical measurements, such as temperature or rainfall values, that the user does not want rescaled. With `preserve_range=True`, the original range of the data will be preserved, the original range of the data will be preserved, even though the output is a float image. Users must then ensure this non-stansard image is properly processed by downstream functions, which may except an mage in [0,1].

In [5]:
from skimage import data
from skimage.transform import rescale
image = data.coins()
image.dtype, image.min(), image.max(), image.shape

rescaled = rescale(image, 0.5)
(rescaled.dtype, np.round(rescaled.min(), 4), np.round(rescaled.max(), 4), rescaled.shape)

rescaled = rescale(image, 0.5, preserve_range=True) 
(rescaled.dtype, np.round(rescaled.min()), np.round(rescaled.max()), rescaled.shape)

(dtype('float64'), 4.0, 232.0, (152, 192))

## Output types

The output type os a function is determined by the function author and is documented for the benefit of the user. While this requires the user to explicitly convert the output to whichever format is needed, it ensures that no unnecessary data copies take place.

A user that requires a specific type of output (e.g., for display purposes), may write:

In [6]:
# from skimage.util import img_as_uint
# out = img_as_uint(sobel(image))
# plt.imshow(out)

## Working with OpenCV

It is possible that you may need to use an image ce=reated using `skimge` with **OpenCV** or vice versa. OpenCV image data can be accessed (without copying) in NumPy (and, thus, in scikit-image). OpenCv uses BGR (instead of scikit-image's RGB) for color images, and its dtype is uint8 by default (See Image data types and what they mean), BGR stands for Blue Green Red.

### Converting BGR to RGB or vice versa

The color images in `skimage` and OpenCV have 3 dimensions: width, height and color. RGB and BGR use the same color space, except the order of colors is reversed.

Note that in `scikit-image` we usually refer to `rows` and `columns` instead of width and height (see Coordinate conventions).

The following instruction effectively reverses the order of the colors, leaving the rows and columns uneffected:

In [7]:
# image = image[:,:,::-1]

### Using an image from OpenCV with `skimage`

If `cv_image` is an array of unsigned bytes, `skimage` will understand it by default. If you prefer working with floating point images, `img_as_float()` can be used to convert the image:

In [8]:
# from skimage.util import img_as_float
# image = img_as_float(any_opencv_image)

### Using an image from `skimage` with OpenCV

The reverse can be achieved with `img_as_ubyte()`:

In [9]:
# from skimage.util import img_as_ubyte
# cv_image = img_as_float(any_skimage)

## Image processing pipeline

This dtypes behavior allows you to string together any `skimage` function without worying about the image dtype. On the other hand, if you want to use a custon function that requires a particular dtype, you should call one of the dtype conversion functions (here, `func1` and `func2` are `skimage` functions:

In [10]:
# from skimage.util import img_as_float
# image = img_as_float(func1(func2(image)))
# processed_image = custom_func(image)

Better yet, you can convert the image internally and use a simplified processing pipeline:

In [11]:
# def custom_func(image):
#    image = img_as_float(image)
#    # do something
#
# processed_image = custom_func(func1(func2(image)))

## Rescaling intensity values

When possible, functions should avoid blindly stretching image intensities (e.g. rescaling a float image so that the min and max intentisties are 0 and 1), since this can healily distort an image. For example, if you're looking for bright markers in dark images, there may be an image where no markers are present; stretching its input intensity to span the full range would make background noise look like markers. 

Sometimes, however, you have images that should span the entire intensity range but do not. For example, some camera store images with 10-, 12-, or 14-bit depth per pixel. If these images are stored in an array with dtype uint16, then the image won't extend over the full intensity range, an thus, would appear dimmer than it should. To correct for this, you can use the `rescale_intensity` function to rescale the image so that it uses the full dtype range:

In [12]:
# from skimage import exposure
# image = exposure.rescale_intensity(img10bit,in_range=(0,2**10-1))

Here, the `in_range` argument is set to the maximum range for a 10-bit image. Be default, `rescale_intensity` stretches the values of `in_range` to match the range of the dtype. `rescale_intensity` also accepts strings as inpits to `in_range` and `out_range`, so the example above could also be written as:

In [13]:
# image = exposure.rescale_intensity(img10bit,in_range='uint10')

## Note about negative values

People very often represent images in signed dtypes, even through they only manipulate the positive values of the image (e.g., using only 0-127 in an int8 image). For this reason, conversion functions *only spread the positive values* of a signed dtype over the entire range of an unsigned dtype. In other words, negative values are clipped to 0 when converting from signed to unsigned dtypes. (Negative values are preserved when converting between signed dtypes.) To prevent this clipping behavior, you should rescale your image beforehand:

In [14]:
# image = exposure.rescale_intensity(img_int32,out_range(0,2**31-1))
# img_uint8 = img_as_ubyte(image)